In [1]:
!pip install pandas

In [2]:
import pandas as pd

#importando os dados
df1 = pd.read_csv("dataset_grupo1.csv", encoding="latin1", sep = ";")

df2 = pd.read_csv("dataset2_grupo1.csv", encoding="latin1", sep = ";")

df3 = pd.read_csv("dataset3_grupo1.csv", encoding="latin1", sep = ";")

#fazendo a junção dos dados em uma tabela única
dados = pd.merge(df1, df2, on="Data")
dados = pd.merge(dados, df3, on="Data")

dados.head()
 
 #conferindo os dados:
print(dados.isnull().sum())
#não há n.a values
print(dados['Data'].value_counts().sort_index())
# dados não repetidos
print(dados.nunique())

#aparentemente tudo certo :)


Data                                                                                                                                     0
433 - Índice nacional de preços ao consumidor-amplo (IPCA) - Var. % mensal                                                               0
3695 - Taxa de câmbio - Livre - Dólar americano (compra) - Fim de período - mensal - u.m.c./US$                                          0
4380 - PIB mensal - Valores correntes (R$ milhões) - R$ (milhões)                                                                        0
4469 - Dívida Líquida do Setor Público - Saldos em R$ milhões - Total - Governo Federal - R$ (milhões)                                   0
4470 - Dívida Líquida do Setor Público - Saldos em R$ milhões - Total - Banco Central - R$ (milhões)                                     0
4472 - Dívida Líquida do Setor Público - Saldos em R$ milhões - Total - Governos estaduais - R$ (milhões)                                0
4473 - Dívida Líquida do Se